<a href="https://colab.research.google.com/github/malcolmrite-dsi/RockVideoClassifier/blob/main/RocksResnetTrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten, Dropout
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.applications import xception
import pandas as pd
import PIL
import matplotlib.pyplot as plt
import os
import shutil

# Training

In [5]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(validation_split=0.2, preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/Module 2 shared folder/samples',
        subset="training",
        seed=3,
        target_size=(64, 64),
        batch_size=64,
        class_mode='categorical')

val_generator = train_datagen.flow_from_directory( '/content/drive/My Drive/Module 2 shared folder/samples', 
        subset="validation",
        seed=3,
        target_size=(64, 64),
        batch_size=64,
        class_mode='categorical')

Found 3132 images belonging to 5 classes.
Found 781 images belonging to 5 classes.


In [11]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/Module 2 shared folder/samples',
        subset="training",
        seed=3,
        target_size=(64, 64),
        batch_size=64,
        class_mode='categorical')

Found 3913 images belonging to 5 classes.


In [6]:
resnet = keras.applications.ResNet50(include_top=False, pooling="max", input_shape=(64,64,3))

94773248/94765736 [==============================] - 1s 0us/step


In [7]:
# mark loaded layers as not trainable
for layer in resnet.layers:
  layer.trainable = False 


In [16]:
data_augmentation = tf.keras.Sequential([
  keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  #keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [9]:
# mark loaded layers as not trainable
#for layer in resnet.layers:
	#layer.trainable = False 

flat = Flatten()(resnet.layers[-1].output)
dense = Dense(1024, activation='relu')(flat)
output = Dense(5, activation='softmax')(dense)
model = Model(inputs=resnet.inputs, outputs=output)

In [ ]:
model.summary()

In [10]:
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(), metrics=["categorical_accuracy"])
checkpoint_best = keras.callbacks.ModelCheckpoint("/content/drive/My Drive/model_best.h5", 
            monitor='loss', verbose=0, save_best_only=True, save_weights_only=False, save_freq='epoch')
checkpoint = keras.callbacks.ModelCheckpoint("/content/drive/My Drive/model_last.h5", 
            verbose=0, save_best_only=False, save_weights_only=False, save_freq='epoch')
model.fit(
        train_generator,
        epochs = 5,
        validation_data=val_generator,
        callbacks=[checkpoint_best]
)

Epoch 1/5
49/49 [==============================] - 921s 19s/step - loss: 1.4298 - categorical_accuracy: 0.7925 - val_loss: 0.6334 - val_categorical_accuracy: 0.8656
Epoch 2/5
49/49 [==============================] - 63s 1s/step - loss: 0.2659 - categorical_accuracy: 0.9119 - val_loss: 0.4664 - val_categorical_accuracy: 0.8707
Epoch 3/5
49/49 [==============================] - 61s 1s/step - loss: 0.1628 - categorical_accuracy: 0.9406 - val_loss: 0.4552 - val_categorical_accuracy: 0.8681
Epoch 4/5
49/49 [==============================] - 62s 1s/step - loss: 0.1231 - categorical_accuracy: 0.9527 - val_loss: 0.4680 - val_categorical_accuracy: 0.8694
Epoch 5/5
49/49 [==============================] - 63s 1s/step - loss: 0.1004 - categorical_accuracy: 0.9623 - val_loss: 0.5946 - val_categorical_accuracy: 0.8438


In [42]:
model.evaluate(val_generator)

13/13 [==============================] - 36s 3s/step - loss: 0.4126 - categorical_accuracy: 0.8822


[0.4126453995704651, 0.8822023272514343]

In [12]:
model.fit(
        train_generator,
        initial_epoch=10,
        epochs = 25,
        validation_data=val_generator, callbacks=[checkpoint, checkpoint_best]
)

Epoch 11/25
62/62 [==============================] - 72s 1s/step - loss: 0.1822 - categorical_accuracy: 0.9443 - val_loss: 0.1983 - val_categorical_accuracy: 0.9283
Epoch 12/25
62/62 [==============================] - 74s 1s/step - loss: 0.1454 - categorical_accuracy: 0.9535 - val_loss: 0.2430 - val_categorical_accuracy: 0.9283
Epoch 13/25
62/62 [==============================] - 76s 1s/step - loss: 0.1018 - categorical_accuracy: 0.9660 - val_loss: 0.0794 - val_categorical_accuracy: 0.9718
Epoch 14/25
62/62 [==============================] - 79s 1s/step - loss: 0.0633 - categorical_accuracy: 0.9785 - val_loss: 0.0779 - val_categorical_accuracy: 0.9731
Epoch 15/25
62/62 [==============================] - 74s 1s/step - loss: 0.0874 - categorical_accuracy: 0.9737 - val_loss: 0.1695 - val_categorical_accuracy: 0.9488
Epoch 16/25
62/62 [==============================] - 78s 1s/step - loss: 0.0601 - categorical_accuracy: 0.9806 - val_loss: 0.0240 - val_categorical_accuracy: 0.9949
Epoch 17/2

In [13]:
model.save("/content/drive/My Drive/model_best.h5")